In [2]:
import sqlite3
import pandas as pd
import csv
from row64tools import ramdb

In [3]:
conn = sqlite3.connect(r"C:\Users\mikha\Downloads\DBexampleROW64.db")
cursor = conn.cursor()

In [4]:
lap_analysis_query = """
SELECT 
    LAPANALYSIS.LapID,
    ANALYSISFORMULATION.Name AS FormulationName,
    ANALYSISCHANNEL.Name AS ChannelName,
    ANALYSISLOCATION.Name AS LocationName,
    ANALYSISCONDITION.Name AS ConditionName,
    LAPANALYSIS.Idx 
FROM LAPANALYSIS
left JOIN ANALYSISFORMULATION ON LAPANALYSIS.FormulationID = ANALYSISFORMULATION.ID
JOIN ANALYSISCHANNEL ON LAPANALYSIS.ChannelID = ANALYSISCHANNEL.ID
JOIN ANALYSISLOCATION ON LAPANALYSIS.LocationID = ANALYSISLOCATION.ID
JOIN ANALYSISCONDITION ON LAPANALYSIS.ConditionID = ANALYSISCONDITION.ID
"""

lap_analysis_df = pd.read_sql_query(lap_analysis_query, conn)

In [9]:
final_query = """
SELECT 
    SEASON.Name AS SeasonName,
    EVENT.Name AS EventName,
    SESSION.Name AS SessionName,
    SESSION.Name || '_' || RUN.Number AS SessionRunNumber,
    LAP.LapNumber,
    RUN.Number AS RunNumber,
    RIDER.Alias AS RiderAlias,
    LAP.ID AS LapID
FROM LAP
JOIN RUN ON LAP.RunID = RUN.ID
JOIN SESSION ON RUN.SessionID = SESSION.ID
JOIN EVENT ON SESSION.EventID = EVENT.ID
JOIN SEASON ON EVENT.SeasonID = SEASON.ID
JOIN RIDER ON RUN.RiderID = RIDER.ID
"""

meta_df = pd.read_sql_query(final_query, conn)

In [10]:
final_lap_analysis_df = meta_df.merge(lap_analysis_df, on="LapID", how="inner")

In [11]:
print(len(lap_analysis_df))

3299165


In [7]:
ramdb.save_from_df(lap_analysis_df, r"C:\Users\mikha\LapAnalysisKTM.ramdb")

In [12]:
final_lap_analysis_df.to_csv(r"C:\Users\mikha\Customers\KTM\final_lap_analysis_df.csv", index=False)